In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
geo=None
#geo='FR'

In [6]:
dbr=pd.read_csv("imdb_dbr.csv")
#if geo!=None:
#    dbr=dbr[dbr.region==geo]
dbr=dbr.drop('Unnamed: 0',axis=1)
dbr=dbr.drop('Unnamed: 0.1',axis=1)
dbr=dbr.drop('Unnamed: 0.1.1',axis=1)
dbr=dbr.drop('Unnamed: 0.1.1.1',axis=1)
dbr=dbr.drop('Unnamed: 0.1.1.1.1',axis=1)


# On se place fin 2012 et sélectionne les acteurs qui ont joué dans au moins 2 films depuis 2008.
start=2008
dbr1=dbr[dbr.startYear>=start]
#on enlève les personnes agées pour éviter l'effet de la mortalité
dbr1=dbr1[dbr1.birthYear>start-70]
# on veut compter seulement sur les 5 années de 2008 à 2012
dbr2=dbr1[dbr1.startYear<start+5]
# on voudra aussi vérifier si les acteurs ont tourné après 2012
dbr3=dbr1[dbr1.startYear>=start+5]
dbr3=dbr3[dbr3.startYear<start+10]

In [7]:
# formons une cohorte avec celles et ceux qui ont tourné au moins deux films dans les 5 années de 2013 à 2017 inclus
list_prov=dbr2.groupby('nconst').count()[['tconst']]
list_prov=list_prov[list_prov.tconst>=2]
cohorte=list_prov.index.tolist()
print(len(cohorte))

8637


In [8]:
#éliminons de nos bases de données les rôles qui ne concernent pas les personnes de notre cohorte
dbr=dbr[dbr.nconst.isin(cohorte)]
dbr2=dbr2[dbr2.nconst.isin(cohorte)]
dbr3=dbr3[dbr3.nconst.isin(cohorte)]

In [9]:
# pour chaque nom de la cohorte, déterminons s'il/elle a ou non tourné ensuite
# pour cela constituons la liste (déjà réduite) des noms présents dans les rôles post 2012
list_2013=dbr3.nconst.unique().tolist()
list_prov['retournera']=list_prov.index.isin(list_2013)

In [10]:
print(list_prov.groupby('retournera').count())

            tconst
retournera        
False         2616
True          6021


40% des acteurs ne retourneront pas de film notable dans les 3 années suivantes et 30% dans les 5 années suivantes.

In [11]:
#ajoutons de nouvelles variables explicatives possibles
new_dat=dbr2.groupby('nconst').agg('mean')
new_dat=new_dat.drop('age',axis=1)
new_dat.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8637 entries, nm0000092 to nm9038038
Data columns (total 5 columns):
ordering         8637 non-null float64
startYear        8637 non-null float64
birthYear        8637 non-null float64
averageRating    8412 non-null float64
numVotes         8412 non-null float64
dtypes: float64(5)
memory usage: 404.9+ KB


In [12]:
# fusionnons avec le dataframe contenant les nombres de films récents
new_dat=pd.concat([new_dat, list_prov], axis=1)

In [13]:
# tconst ici indique le nombre de films tournés durant les 5 dernières années. Changeons pour un nom plus explicite
new_dat.rename(columns={'tconst':'Num5Y'}, inplace=True)

nous avons ici un certain nombre de stats sur les derniers films (nombre, moyenne des rang des roles, des années, des ratings qui peuvent servir de variables explicatives

In [14]:
# ajoutons y le nombre total de films de la carrière jusqu'ici
tot_films=dbr.groupby('nconst').agg('count')['tconst']
new_dat=pd.concat([new_dat, tot_films], axis=1)

In [15]:
new_dat.head(5)

,ordering,startYear,birthYear,averageRating,numVotes,Num5Y,retournera,tconst
nconst,,,,,,,,
nm0000092,2.500000,2010.500000,1939.0,6.450000,20897.500000,2,False,15
nm0000093,1.000000,2009.833333,1963.0,7.283333,422899.666667,6,True,32
nm0000096,3.500000,2012.000000,1968.0,6.650000,8615.000000,2,True,13
nm0000098,1.428571,2009.571429,1969.0,6.171429,98413.142857,7,True,27
nm0000099,2.000000,2010.000000,1968.0,5.850000,3118.500000,2,True,21


In [16]:
# reste à ajouter des variables textuelles correspondant aux regions, professions etc.. les plus fréquentes pour un acteur/trice
# malheureusement comme je n'ai pas réussi à faire d'aggrégation avec 'mode' je suis obligé de briser l'interdit 
# et parcourir un dataframe (il doit falloir utiliser "apply", je regarderais plus tard comment faire
Cate={}
Genr={}
Reg={}
prProf={}
for nom in cohorte:
    buff=dbr2[dbr2.nconst==nom]
    Cate[nom]=buff.category.mode()[0]
    Genr[nom]=buff.genres.mode()[0]
    Reg[nom]=buff.region.mode()[0]
    prProf[nom]=buff.primaryProfession.mode()[0]


In [14]:
# faisons en un dataframe qu'on fusionnera avec le précédent
SCate=pd.Series(Cate)
SGenr=pd.Series(Genr)
SReg=pd.Series(Reg)
SProf=pd.Series(prProf)

In [15]:
new_dat=pd.concat([new_dat, SCate], axis=1)
new_dat.rename(columns={0:'SCate'}, inplace=True)

In [16]:
new_dat=pd.concat([new_dat, SGenr], axis=1)
new_dat.rename(columns={0:'SGenr'}, inplace=True)
new_dat=pd.concat([new_dat, SReg], axis=1)
new_dat.rename(columns={0:'SReg'}, inplace=True)
new_dat=pd.concat([new_dat, SProf], axis=1)
new_dat.rename(columns={0:'SProf'}, inplace=True)

In [17]:
new_dat.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8637 entries, nm0000092 to nm9038038
Data columns (total 12 columns):
ordering         8637 non-null float64
startYear        8637 non-null float64
birthYear        8637 non-null float64
averageRating    8412 non-null float64
numVotes         8412 non-null float64
Num5Y            8637 non-null int64
retournera       8637 non-null bool
tconst           8637 non-null int64
SCate            8637 non-null object
SGenr            8637 non-null object
SReg             8637 non-null object
SProf            8637 non-null object
dtypes: bool(1), float64(5), int64(2), object(4)
memory usage: 818.2+ KB


In [18]:
#Sauvegardons ces données
new_dat.to_excel('cohorte.xlsx')